In [ ]:
import sys
# import pyccl as ccl
import pickle
import camb
from skylens import *
from survey_utils import *
import seaborn as sns
colors=sns.color_palette()
markers=['o','s','^','>','v']

In [ ]:
ncpu=2
LC,scheduler_info=start_client(Scheduler_file=None,local_directory=None,ncpu=None,n_workers=ncpu,threads_per_worker=1,
                              memory_limit='120gb',dashboard_address=8811,processes=True)
client=client_get(scheduler_info=scheduler_info)

In [ ]:
s1_s2s={}
s1_s2s[corr_gg]=(0,0)
s1_s2s[corr_ll]=[(2,2),(2,-2)]
s1_s2s[corr_ggl]=(0,2)

In [ ]:
wigner_files={}
home='/verafs/scratch/phy200040p/sukhdeep/physics2/skylens/'
wig_home=home+'temp/'
wigner_files[0]= wig_home+'/dask_wig3j_l2200_w4400_0_reorder.zarr'
# wigner_files[0]= wig_home+'/dask_wig3j_l3500_w2100_0_reorder.zarr'
wigner_files[2]= wig_home+'/dask_wig3j_l2200_w4400_2_reorder.zarr'

wigner_step=100

In [ ]:
test=False

In [ ]:
nside=1024
if test:
    nside=128
window_lmax=nside*2
use_window=True
unit_window=True
f_sky=0.3

do_pseudo_cl=True
do_xi=True

bin_cl=True
bin_xi=True

use_binned_l=True
use_binned_theta=True

store_win=True

nz_PS=50

do_cov=True
SSV_cov=False
tidal_SSV_cov=False
Tri_cov=False

In [ ]:
z0=1 #1087
# zs_bin1=source_tomo_bins(zp=[z0],p_zp=np.array([1]),ns=30,use_window=use_window,nside=nside)
shear_zbins=lsst_source_tomo_bins(nbins=1,use_window=use_window,nside=nside,f_sky=f_sky,unit_window=unit_window)
galaxy_zbins=shear_zbins

In [ ]:
corrs=[corr_gg]#corr_ll,corr_gg,corr_ggl]

In [ ]:
bi=(0,0)
corr=corrs[0]
s=s1_s2s[corr]
if corr==corr_ll:
    s=(2,2)

In [ ]:
#setup parameters
lmax_cl=nside #2000
lmin_cl=0
l0=np.arange(lmin_cl,lmax_cl)

lmin_cl_Bins=lmin_cl+2
lmax_cl_Bins=lmax_cl-10
Nl_bins=50
if test:
    Nl_bins=10
l_bins=get_l_bins(l_min=lmin_cl_Bins,l_max=lmax_cl_Bins,N_bins=Nl_bins,binning_scheme='log',min_modes=50)#,binning_mode='decreasing')
l_bins=l_bins[l_bins<=lmax_cl_Bins]
# l_bins=np.unique(np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins+1)))
lb=.5*(l_bins[1:]+l_bins[:-1])
l_bins_center=lb*1
Nl_bins=len(lb)

# l=np.unique(np.int64(np.logspace(np.log10(lmin_cl),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell
l=l0

In [ ]:
theta_min=0.01/60
# theta_min=hp.nside2resol(512, arcmin = True)/60 #100/nside #1./60
theta_max=600./60
n_theta_bins=151
# theta_bins,th=get_theta_bins(theta_min=theta_min,theta_max=theta_max,N_bins=n_theta_bins,binning_scheme='log',n_theta_samples=40)#,max_modes=1)
theta_bins=np.logspace(np.log10(theta_min),np.log10(theta_max),n_theta_bins+1)
th=np.logspace(np.log10(theta_min),np.log10(theta_max),n_theta_bins*40)
theta_bins_center=0.5*(theta_bins[1:]+theta_bins[:-1])
thb=theta_bins_center*1
n_theta_bins=len(thb)

#Hankel Transform setup
WT_kwargs={'l':l0,'l_cut_weights':np.ones_like(l0),'theta':th*d2r,'s1_s2':[(2,2),(2,-2),(0,0)],
           'wig_d_taper_order_low':16,'wig_d_taper_order_high':20}
# if globals().get('WT') is None:
WT=wigner_transform(**WT_kwargs )
# WT_b=wigner_transform(**WT_kwargs)
WT_b=WT
WT.gather_data()

In [ ]:
client

In [ ]:
Skylens_kwargs=parse_dict(locals())
kappa_xi0=Skylens(**Skylens_kwargs)

Skylens_kwargs_nW=copy.deepcopy(Skylens_kwargs)
Skylens_kwargs_nW['use_window']=False
kappa_xi0_nW=Skylens(**Skylens_kwargs_nW)

In [ ]:
cl_bu=client.gather(kappa_xi0.cl_bin_utils)
xi_bu=client.gather(kappa_xi0.xi_bin_utils)

In [ ]:
%time xiHG0=kappa_xi0.xi_tomo()
%time xiH0=xiHG0['stack'].compute()
%time xi0=client.compute(xiHG0['xi']).result()

In [ ]:
%time xiHG0_nW=kappa_xi0_nW.xi_tomo()
%time xiH0_nW=xiHG0_nW['stack'].compute()
%time xi0_nW=client.compute(xiHG0_nW['xi']).result()

In [ ]:
%time clG0_nW=kappa_xi0_nW.cl_tomo()
%time cl0_nW=client.compute(clG0_nW['cl_b']).result()
%time pcl0_nW=client.compute(clG0_nW['stack']).result()

In [ ]:
%time clG0=kappa_xi0.cl_tomo()
%time cl0=client.compute(clG0['cl_b']).result()
%time pcl0=client.compute(clG0['stack']).result()
# %time pcl0_cov=client.compute(clG0['cov']).result()

# %time cl0_0=client.compute(clG0['cl']).result()

In [ ]:
win=kappa_xi0.Win

In [ ]:
M=win['cl'][corr][bi]['M']
Mi=np.linalg.pinv(M)
Wi=kappa_xi0.inv_WT_binned[corr][s][bi].result()

In [ ]:
def matrix_cut(mat=[],x=[]):
    m=mat[x]
    N=sum(x)
    m2=np.zeros((N,N))
    j=0
    for i in m:
        m2[j]=i[x]
        j=j+1
    return m2

def SN_cum(cov=[],lb=[],cl=[],diag=False,lmin=2,lmax=1e4,use_hartlap=False,nsim=1000):
    sni=np.zeros_like(lb)
    for i in np.arange(len(lb)):
        if lb[i]<lmin or lb[i]>lmax:
            continue
        x=lb<=lb[i]
        x*=lb>lmin
        cov2_cut=matrix_cut(mat=cov,x=x)
        if diag:
            cov2_cut=np.diag(np.diag(cov2_cut))
        cov2_cut_inv=np.linalg.inv(cov2_cut)
        
        cl_i=cl[x]
        SN2=cl_i@cov2_cut_inv@cl_i
        if use_hartlap:
            SN2*=(nsim-2-x.sum())/(nsim-1)
        sni[i]=SN2
    return sni

In [ ]:
cl_pcl=Mi@pcl0['pcl_b']
cl_pcl_cov=Mi@pcl0['cov']@Mi.T
cl_xi=Wi@xiH0['xi']
cl_xi_cov=Wi@xiH0['cov']@Wi.T

In [ ]:
cl=pcl0_nW['pcl_b']
cl_cov=pcl0_nW['cov']

In [ ]:
cl_pcl_SN=SN_cum(cov=cl_pcl_cov,lb=lb,cl=cl_pcl,lmin=10)
cl_xi_SN=SN_cum(cov=cl_xi_cov,lb=lb,cl=cl_xi,lmin=10)

cl_SN=SN_cum(cov=cl_cov,lb=lb,cl=cl,lmin=10)

In [ ]:
plot(lb,cl_xi_SN)
plot(lb,cl_pcl_SN,'--')
plot(lb,cl_SN,':')
loglog()

In [ ]:
plot(lb,cl0[corr][bi])
plot(lb,cl,':')
plot(lb,Mi@pcl0['pcl_b'])
plot(lb,Wi@xiH0['xi'])
loglog()

In [ ]:
lb